# Final Project. Text categorization with NLTK Reuters corpus.
First step will be to download all the necesary libraries and classes.

In [ ]:
import nltk
from nltk.corpus import reuters
from nltk import FreqDist, bigrams
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
import random
from collections import Counter

# Download necessary resources
nltk.download('reuters') # Fetches the Reuters newswire articles so we can load words, file ids, and categories
nltk.download('punkt') # Installs the Punkt tokenizer models for sentence/word tokenization

# 1. Data partitioning. Determenistic 70/10/20 sets

In [3]:
# splitting the data and determening the ids for documents from each data set. 
# Code partially taken and adapted from Homework assignment 3.4
fileids = sorted(reuters.fileids())
total = len(fileids)
test_size = int(0.2 * total)
val_size  = int(0.1 * total)

test_ids  = fileids[:test_size] # first 20% of sorted IDs is test set
val_ids   = fileids[test_size:test_size + val_size] # next 10% is validation
train_ids = fileids[test_size + val_size:]# last 70% is training set

## 2. Loading Texts and Gold-Standard Labels

In this step we convert our document ID lists into the actual data our classifier will see:

1. **Token lists**: the raw word sequence for each document.  
2. **Gold labels**: the primary Reuters category for each document.

We build three lists—`train_docs`, `val_docs`, and `test_docs`—each containing `(word_list, label)` tuples. Finally, we print the counts to verify our split sizes.


In [ ]:
# Load texts and labels. first create a list , then append a "words,label" tupple to a list. 
# we do that for train, validation and test docs. For better readability print the amount of docs from each category. 
train_docs = []
for fid in train_ids:
    words = list(reuters.words(fid)) #method taken from https://www.nltk.org/book/ch02.html
    label = reuters.categories(fid)[0]#method taken from https://www.nltk.org/book/ch02.html
    train_docs.append((words, label))

val_docs = []
for fid in val_ids:
    words = list(reuters.words(fid))
    label = reuters.categories(fid)[0]
    val_docs.append((words, label))

test_docs = []
for fid in test_ids:
    words = list(reuters.words(fid))
    label = reuters.categories(fid)[0]
    test_docs.append((words, label))

print(f"Training docs:   {len(train_docs)}")
print(f"Validation docs: {len(val_docs)}")
print(f"Test docs:       {len(test_docs)}")


## 3. Verification of Category Proportions
In this step we check that each data split (training, validation, test) preserves the overall
distribution of Reuters categories. We count how many documents of each label appear in each
partition. This ensures our deterministic slicing
did not skew the data.

In [ ]:
#this function counts amount of docs in each split
def category_counts(docs):
    counts = Counter()
    for _, label in docs:
        counts[label] += 1
    return counts

#Compute label counts for overall corpus + each split
overall_counts = category_counts(train_docs + val_docs + test_docs)
train_counts   = category_counts(train_docs)
val_counts     = category_counts(val_docs)
test_counts    = category_counts(test_docs)

#Display raw counts for overall corpus
print(f"Overall ({len(train_docs) + len(val_docs) + len(test_docs)} docs):")
for cat, cnt in overall_counts.most_common(5): #most_common method taken from https://www.nltk.org/book/ch02.html
    print(f"  {cat}: {cnt}")

#Display raw counts for training split
print(f"\nTraining ({len(train_docs)} docs):")
for cat, cnt in train_counts.most_common(5):
    print(f"  {cat}: {cnt}")

#Display raw counts for validation split
print(f"\nValidation ({len(val_docs)} docs):")
for cat, cnt in val_counts.most_common(5):
    print(f"  {cat}: {cnt}")

# tDisplay raw counts for test split
print(f"\nTest ({len(test_docs)} docs):")
for cat, cnt in test_counts.most_common(5):
    print(f"  {cat}: {cnt}")


## 4. Feature Extraction and Building Extractors

In this section we generate our unigram and bigram feature sets from the training data, then define three extractor functions:

1. contains_features – presence of each top-N unigram  
2. bigram_features_extractor – presence of each top-N bigram  
3. combined_features – merges unigram and bigram indicators into one feature dict

These functions will turn a raw token list into the (feature_dict, label) pairs needed by our Naive Bayes classifier.


In [6]:
# Feature Extraction: build unigram & bigram feature lists using FreqDist and slicing. Adapted from homework 2.3 
all_words = FreqDist(w.lower() for fid in train_ids for w in reuters.words(fid))
word_features = list(all_words)[:2000]

all_bigrams = FreqDist(bg for fid in train_ids for bg in bigrams(w.lower() for w in reuters.words(fid)))
bigram_features = list(all_bigrams)[:5000]

# Feature extractors
def contains_features(document, word_features):
    doc_words = set(w.lower() for w in document)
    features = {}
    for word in word_features:
        features[f"contains({word})"] = (word in doc_words)
    return features

def bigram_features_extractor(document, bigram_features):
    lower = [w.lower() for w in document]
    doc_bi = set(bigrams(lower))
    features = {}
    for bm in bigram_features:
        features[f"bigram({bm[0]}_{bm[1]})"] = (bm in doc_bi)
    return features

def combined_features(document, word_features, bigram_features):
    feats = contains_features(document, word_features)               # unigram flags
    more  = bigram_features_extractor(document, bigram_features)     # bigram flags
    feats.update(more)  # merge bigram entries into the feats dict
    return feats


# 5.Prepare feature-label pairs
In order to train and evaluate an NLTK classifier, we need each example as a tuple of (feature_dictionary, label)
where feature_dictionary maps feature names to Boolean values (True/False) and label is the gold category. The code below builds six such lists—one for each combination of split (train/validation/test) and feature scheme (unigram vs. unigram+bigram).

In [7]:
#EXPLAIN
dataset_uni = []
for doc, label in train_docs:
    feats = contains_features(doc, word_features)
    dataset_uni.append((feats, label))

dataset_bi = []
for doc, label in train_docs:
    feats = combined_features(doc, word_features, bigram_features)
    dataset_bi.append((feats, label))

val_uni = []
for doc, label in val_docs:
    feats = contains_features(doc, word_features)
    val_uni.append((feats, label))

val_bi = []
for doc, label in val_docs:
    feats = combined_features(doc, word_features, bigram_features)
    val_bi.append((feats, label))

test_uni = []
for doc, label in test_docs:
    feats = contains_features(doc, word_features)
    test_uni.append((feats, label))

test_bi = []
for doc, label in test_docs:
    feats = combined_features(doc, word_features, bigram_features)
    test_bi.append((feats, label))


## 6. Baseline: Most-Frequent-Class

Here we implement a trivial classifier that always predicts the single most common label from the training data.
This “majority-class” baseline establishes a floor for performance—any real classifier should beat it and optimally- by a good margin.

In [ ]:
#Count how often each label occurs in the training set
freq_counter = Counter()
for _, lbl in dataset_uni: # dataset_uni is list of (feature_dict, label) for training
    freq_counter[lbl] += 1 # increment the count for this label
freq_label = freq_counter.most_common(1)[0][0] #.most_common(1) returns a list with one (label, count) pair

# 6.3 Predict that label for every test document and count how many are correct
correct = 0
total_test = len(test_docs) # number of documents in the test split
for _, lbl in test_uni:  # test_uni is list of (feature_dict, label) for testing
    if lbl == freq_label: # if the true label matches our “always-predict” label
        correct += 1  # it’s a correct prediction
        
baseline_acc = correct / total_test
print(f"Most-Frequent-Class Baseline Accuracy: {baseline_acc:.4f}")

# 7. Model Training and Validation (Naive Bayes Only)
In this step we train two Naive Bayes classifiers—one using unigram features only and one using the combined unigram+bigram features—and evaluate both on the held-out validation set to choose the better feature scheme before final testing.

In [ ]:
#adapted from homeassignment 2.3 
nb_uni = NaiveBayesClassifier.train(dataset_uni)
nb_bi  = NaiveBayesClassifier.train(dataset_bi)

# Compute validation accuracies using NLTK's built-in accuracy helper
from nltk.classify.util import accuracy

tacc_uni = accuracy(nb_uni, val_uni)
tacc_bi  = accuracy(nb_bi,  val_bi)
print("Validation Accuracy Results:")
print(f"  Unigram only      : {tacc_uni:.4f}")
print(f"  Unigram+Bigram    : {tacc_bi:.4f}")


# 8. Final Evaluation
In this final step we:

1. Select the better‐performing feature scheme based on validation accuracy.  
2. Evaluate the chosen model on the held‐out test set to get its overall accuracy.  
3. Compute detailed per-class precision and recall to see which categories the model handles well or poorly.

In [ ]:
#Choose the best classifier based on validation accuracies

if tacc_bi > tacc_uni:
    best_clf  = nb_bi # pick the unigram+bigram model
    best_feats = True # flag indicating we use the combined features
    scheme    = 'Unigram+Bigram'
else:
    best_clf  = nb_uni # pick the unigram-only model
    best_feats = False # flag for unigram features only
    scheme    = 'Unigram only'
print(f"Best feature scheme: {scheme}")

#Prepare the test set corresponding to the chosen scheme
test_set = test_bi if best_feats else test_uni

from nltk.classify.util import accuracy as nltk_accuracy
# Compute overall test accuracy
test_acc = nltk_accuracy(best_clf, test_set)
print(f"Test Accuracy ({scheme}): {test_acc:.4f}")

# Compute per-class precision and recall
tp = Counter() # true positives per label
fp = Counter() # false positives per label
fn = Counter() # false negatives per label
labels = set() # to collect all labels we see

# 8.5 Loop over each test example
for feats, lbl in test_set:
    pred = best_clf.classify(feats) # models predicted label
    labels.add(lbl) # record the true label
    labels.add(pred) # record the predicted label
    if pred == lbl:
        tp[lbl] += 1  # correct prediction so true positive for that label
    else:
        fp[pred] += 1 # predicted “pred” but true was something else sp false positive for “pred”
        fn[lbl] += 1 # true was “lbl” but predicted something else so false negative for “lbl”

print("\nPer-class precision and recall:")
for lbl in sorted(labels): # iterate labels in alphabetical order
    tp_val = tp[lbl]
    fp_val = fp[lbl]
    fn_val = fn[lbl]
    # Avoid division by zero: if no predictions/true examples, set metric to 0.0
    prec = tp_val / (tp_val + fp_val) if (tp_val + fp_val) > 0 else 0.0
    rec  = tp_val / (tp_val + fn_val) if (tp_val + fn_val) > 0 else 0.0
    print(f"{lbl:15s} Prec: {prec:.2f}  Rec: {rec:.2f}")


## 3. Final Project Report

### 3.1 Task Definition and Motivation  
I build a text categorization system that assigns each Reuters news article to one of topic labels (e.g. `earn`, `acq`, `crude`, `grain`, …). We did a similar sentiment analysis (positive/negative), but this time it is a multi-class problem over dozens of economic categories.  
- Why it is interesting to me:
1. I really enjoyed the homework assignment with sentiment analysis and i saw this project topic as a chance to go further with text labeling.  
2. I wanted to exercise high-dimensional text features and probabilistic modeling.  
3. I think this is also a good mini project for my personal portfolio, of course like any other project on the list of topics, but this one just got my attention straight away. 

### 3.2 Data
Like with any NLP project I need to decide what data will be used and how it will be used. 
I used NLTK’s built-in Reuters corpus, which contains around 10700 newswire articles labeled with one or more topic tags. 
According to the commentary given to me by Mathias, we do determenistic data split. Why? In short, deterministic partitioning makes experimental results stable, comparable, and trustworthy. By definition deterministic data split means that every time the code is run, we end up with exactly the same train/validation/test partitions—there’s no randomness. In case of this particular project we achieve this by:
- Sorting all document IDs in a fixed order (alphabetically).
- Slicing that sorted list at the 20% and 30% marks to carve out test, validation, and training IDs. More details below.

- Deterministic 70/10/20 split:  
  1. Test set (20%) – first 20% of sorted file IDs  
  2. Validation set (10%) – next 10%  
  3. Training set (70%) – remaining 70%
- Sorting + fixed slicing ensures the same split every run while approximately preserving the global label distribution.  
- Gold standard: each article’s first (primary) Reuters category.

### 3.3 Machine Learning Algorithm  
In this particular project I employ supervised learning using Naive Bayes from NLTK:  
- Why Naive Bayes?  
  - I think it is well-suited to high-dimensional, bag-of-words features.  
  - It is fast to train and easy to interpret.  
  - Provides strong baselines in text classification tasks.
  - Supervised learning. 

### 3.4 Feature Engineering  
We extract two sets of features from the training split:  
1. Unigram bag-of-words – the top or 2 000 most frequent words.  
2. Unigram+bigram – same top N unigrams plus the top M most frequent consecutive word-pairs (bigrams). So although unigrams and bigrams represent different n-gram orders, they are united into one flat feature space with each feature distinctly and simply merging them into a single dictinoary. That lets us directly compare model A with only word-presence signals versus model B with both word-presence and phrase-presence signals on exactly the same classification task. 

Each document is converted into a feature dictionary mapping
contains(word) → True/False
bigram(word1_word2) → True/False

These boolean indicators feed directly into the Naive Bayes classifier.

Why combining uni and bi? I came up with this idea quite spontaneously- I wanted to make something different from the homeworks that we did. For that reason I decided to check what will happen if we combine "best of both worlds".

### 3.5 Baseline Evaluation
For this project i implemented majority class baseline: always predict the single most frequent training label (e.g. `earn`). So we take the most common label out of all and evaluate the accuracy would we apply the same category to all the documents. This Baseline accuracy is about 30.83% on the test set. Why setting the baseline? This sets the floor or a minimun limit: our model must exceed this to prove it learns real patterns.

### 3.6 Validation and Model Selection  
On the validation set we compare:  
- NB + unigrams only  
- NB + unigrams+bigrams

Out of these two we choose the best one (turned out to be simpler unigram-only model) for final evaluation.

### 3.7 Final Test Evaluation  
We evaluate the chosen model on the held-out test set where we achieve overall accuracy: ~76.03% which is well above baseline.  
Per-class precision/recall: 
  - Best recognized: `earn` (Prec 0.83, Rec 0.90), `acq` (0.84/0.84)  
  - Mid-frequency: `crude`, `grain`, `coffee` around 0.50–0.60  
  - Hardest (rare classes): many with 0.00 precision/recall due to few training examples  

This quantitative analysis shows strengths on well-represented topics and at the same time presents under learned categories which model failed to generalize from. 


### 3.8 Overfitting and Generalization  
No signs of overfitting: training and validation accuracies are comparable (76%), and test accuracy (69%) only dips moderately.  
As a part of prevention strategy we could limit feature size.

### 3.9 Conclusions and Future Work  
Naive Bayes system outperforms the trivial baseline by a large margin. Especially unigrams capture most topic-signal and combined uni+bi was identical. Reason for them being identical might be several:
- mistake in my code, although i was not able to identify one. 
- Low bigram coverage: Many of top 2 000 bigrams may be too rare to appear often in validation, so they don’t affect scores.
- Dominant unigram signals: the most predictive single words might already give such a strong tilt toward the correct label that adding phrase‐level flags adds no new information.
- If we increase the increase the bigram_features up to 5000 and leave unigram at 2000 then validation accuracy difference is still minimal:
  Unigram only      : 0.7607
  Unigram+Bigram    : 0.7681

Best predicted class was `earn`; hardest classes were any low-frequency commodities.

Possibly in the future i could experiment with additional classifiers, use metadata features (e.g. document length, section headers), or possibly address rare-class performance through hierarchical grouping of related topics.